In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle  # To save the tokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Path to your file in Drive (Change "MyDrive" and filename accordingly)
file_path = "/content/drive/MyDrive/61262-0.txt"

# Open and read the file
with open(file_path, 'r') as file:
    content = file.read()

# Print the content
#print(content)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the text
with open('/content/drive/MyDrive/61262-0.txt', 'r', encoding='utf-8') as f:
    text = f.read().lower()

print(f"Total characters in text: {len(text)}")
print(f"Sample text:\n{text[:500]}...")

Total characters in text: 296991
Sample text:
*** start of the project gutenberg ebook 61262 ***

  poirot investigates




  by the same author

  the mysterious affair at styles

  the secret adversary

  the murder on the links

  the bodley head




  poirot investigates

  by agatha christie




  london

  john lane the bodley head limited




  first published in great britain by
  john lane company, the bodley head limited, 1924

  copyright © 1924 agatha christie limited




  contents


  i the adventure of “the western star”

  i...


In [ ]:
%pip install nltk

In [ ]:
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1  # +1 for out-of-vocabulary words

print(f"Total unique words: {total_words}")

Total unique words: 6701


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
input_sequences = []
for line in text.split('\n'):  # Split by paragraphs
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences to equal length
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Split into X (features) and y (labels)
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

print(f"Input shape: {X.shape}, Output shape: {y.shape}")

Input shape: (49313, 16), Output shape: (49313, 6701)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
model = Sequential([
    Embedding(total_words, 128, input_length=max_sequence_len-1),  # -1 because we truncated the last word
    LSTM(256, return_sequences=True),  # Stacked LSTM
    Dropout(0.3),
    LSTM(128),
    Dense(128, activation='relu'),
    Dense(total_words, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X, y,
    epochs=100,
    batch_size=64,
    validation_split=0.2
)

Epoch 1/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 112s 182ms/step - accuracy: 0.0581 - loss: 6.3277 - val_accuracy: 0.0704 - val_loss: 6.6003
Epoch 2/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 136s 173ms/step - accuracy: 0.0667 - loss: 6.0603 - val_accuracy: 0.0752 - val_loss: 6.5719
Epoch 3/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 150s 186ms/step - accuracy: 0.0792 - loss: 5.8479 - val_accuracy: 0.0814 - val_loss: 6.5904
Epoch 4/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 144s 190ms/step - accuracy: 0.0936 - loss: 5.6387 - val_accuracy: 0.0878 - val_loss: 6.6301
Epoch 5/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 140s 187ms/step - accuracy: 0.1020 - loss: 5.4831 - val_accuracy: 0.0935 - val_loss: 6.7534


In [ ]:
import pickle
model.save('poirot_language_model.h5')
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
import numpy as np
def generate_text(seed_text, num_words=20, temperature=1.0):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]

        # Adjust randomness with temperature
        predicted_probs = np.log(predicted_probs) / temperature
        exp_probs = np.exp(predicted_probs)
        predicted_probs = exp_probs / np.sum(exp_probs)

        predicted_id = np.random.choice(range(total_words), p=predicted_probs)
        output_word = tokenizer.index_word.get(predicted_id, '')  # Handle unknown words
        seed_text += " " + output_word
    return seed_text

# Example usage
print(generate_text("Poirot examined", temperature=0.7))

Poirot examined you be seen i are got that placidly he know me ” he had be too all there at the


In [ ]:
# demo.py
import tensorflow as tf
import pickle
import numpy as np

# Load model and tokenizer
model = tf.keras.models.load_model('poirot_language_model.h5')
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

seed = input("Enter starting words (e.g., 'Poirot said'): ")
print(generate_text(seed, temperature=0.7))

Enter starting words (e.g., 'Poirot said'): I say
I say it i be get it and loud is a tent he was live that ” the facts of a pocket


In [ ]:
!pip install --upgrade gradio  # Ensure Gradio is up-to-date
import gradio as gr
import tensorflow as tf
import pickle
import numpy as np

# Load your model and tokenizer
model = tf.keras.models.load_model('poirot_language_model.h5')
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Get total words (needed for generation)
total_words = len(tokenizer.word_index) + 1
max_sequence_len = 20  # Adjust if necessary

# Your generate_text function
def generate_text(seed_text, num_words=20, temperature=1.0):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]

        predicted_probs = np.log(predicted_probs) / temperature
        exp_probs = np.exp(predicted_probs)
        predicted_probs = exp_probs / np.sum(exp_probs)

        predicted_id = np.random.choice(range(total_words), p=predicted_probs)
        output_word = tokenizer.index_word.get(predicted_id, '')
        seed_text += " " + output_word
    return seed_text

# Create the Gradio interface
iface = gr.Interface(
    fn=generate_text,
    inputs=gr.Textbox(lines=2, placeholder="Enter starting words (e.g., 'Poirot said')"),
    outputs="text",
    title="Poirot Text Generator Based on",
    description="Generate text in the style of Agatha Christie's Hercule Poirot."
)

# Launch the interface
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0ea00376e2596ae988.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
%pip install pyngrok --quiet

In [ ]:
import gradio as gr
import tensorflow as tf
import pickle
import numpy as np
from pyngrok import ngrok
from IPython.display import display, HTML

# Load your model and tokenizer
model = tf.keras.models.load_model('poirot_language_model.h5')
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Get total words (needed for generation)
total_words = len(tokenizer.word_index) + 1
max_sequence_len = 20  # Adjust if necessary

# Your generate_text function
def generate_text(seed_text, num_words=20, temperature=1.0):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]

        predicted_probs = np.log(predicted_probs) / temperature
        exp_probs = np.exp(predicted_probs)
        predicted_probs = exp_probs / np.sum(exp_probs)

        predicted_id = np.random.choice(range(total_words), p=predicted_probs)
        output_word = tokenizer.index_word.get(predicted_id, '')
        seed_text += " " + output_word
    return seed_text


def generate_text_with_comparisons(seed_text, num_words=20, temperature=1.0):
    # Get output from your model
    your_model_output = generate_text(seed_text, num_words, temperature)

    # Placeholder for external LLM outputs (Gemini, ChatGPT, etc.)
    # Replace with actual API calls and authentication
    gemini_output = "Gemini API call here... (using seed_text)"
    chatgpt_output = "ChatGPT API call here... (using seed_text)"

    return your_model_output, gemini_output, chatgpt_output

iface = gr.Interface(
    fn=generate_text_with_comparisons,
    inputs=gr.Textbox(lines=2, placeholder="Enter starting words (e.g., 'Poirot said')"),
    outputs=[
        gr.Textbox(label="Your Model Output"),
        gr.Textbox(label="Gemini Output"),
        gr.Textbox(label="ChatGPT Output"),
    ],
    title="LSTM-Based Agatha Christie Style Text Generator",
    description="Generate text and compare with other LLMs."
)
# Launch the interface
iface.launch(inbrowser=True, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5304c9860f229a6736.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**NEWLY TRAINED MODEL WITH MORE EPOCHS**

In [1]:
import torch
import os

# Define model and optimizer
from models import YourModel  # Replace with your actual model class
model = YourModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

# Path to saved checkpoint
checkpoint_path = "your_model_checkpoint.pth"

# Check if a checkpoint exists
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    start_epoch = checkpoint["epoch"] + 1  # Resume from next epoch
    print(f"Resuming training from epoch {start_epoch}")
else:
    start_epoch = 0  # Train from scratch

# Set total epochs (add new ones if continuing)
num_new_epochs = 50  # Number of extra epochs
num_epochs = start_epoch + num_new_epochs

# Training loop
for epoch in range(start_epoch, num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(batch)
        loss = loss_function(outputs, batch)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch}/{num_epochs}] - Loss: {loss.item()}")

    # Save model checkpoint
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
    }, checkpoint_path)


ModuleNotFoundError: No module named 'models'